<a href="https://colab.research.google.com/github/balakumar-dataengineer/testrepo/blob/master/Pyspark_Column_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('Pyspark') \
                          .master('local') \
                          .getOrCreate()

print(spark)

In [3]:
data=[(101, "Alice", ["Laptop", "Mouse"], 1200, "2024-02-15", "Shipped"),
    (102, "Bob", ["Phone"], 800, "2024-02-16", "Pending"),
    (103, "Charlie", ["Tablet", "Keyboard"], 600, "2024-02-17", "Delivered"),
    (104, "David", ["Monitor"], 300, "2024-02-18", "Canceled"),
    (105, "Emma", ["Headphones", "Charger"], 150, "2024-02-19", "Shipped")]

schema=['id','name','items','amount','orderDate','orderStatus']

df = spark.createDataFrame(data, schema)
df.show()
df.printSchema()

+---+-------+--------------------+------+----------+-----------+
| id|   name|               items|amount| orderDate|orderStatus|
+---+-------+--------------------+------+----------+-----------+
|101|  Alice|     [Laptop, Mouse]|  1200|2024-02-15|    Shipped|
|102|    Bob|             [Phone]|   800|2024-02-16|    Pending|
|103|Charlie|  [Tablet, Keyboard]|   600|2024-02-17|  Delivered|
|104|  David|           [Monitor]|   300|2024-02-18|   Canceled|
|105|   Emma|[Headphones, Char...|   150|2024-02-19|    Shipped|
+---+-------+--------------------+------+----------+-----------+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- amount: long (nullable = true)
 |-- orderDate: string (nullable = true)
 |-- orderStatus: string (nullable = true)



In [10]:
from pyspark.sql.functions import col

df.select(df.name,df.items).show()

df.select(df['name'],df['items']).show()

df.select(col('name'),col('items')).show()

+-------+--------------------+
|   name|               items|
+-------+--------------------+
|  Alice|     [Laptop, Mouse]|
|    Bob|             [Phone]|
|Charlie|  [Tablet, Keyboard]|
|  David|           [Monitor]|
|   Emma|[Headphones, Char...|
+-------+--------------------+

+-------+--------------------+
|   name|               items|
+-------+--------------------+
|  Alice|     [Laptop, Mouse]|
|    Bob|             [Phone]|
|Charlie|  [Tablet, Keyboard]|
|  David|           [Monitor]|
|   Emma|[Headphones, Char...|
+-------+--------------------+

+-------+--------------------+
|   name|               items|
+-------+--------------------+
|  Alice|     [Laptop, Mouse]|
|    Bob|             [Phone]|
|Charlie|  [Tablet, Keyboard]|
|  David|           [Monitor]|
|   Emma|[Headphones, Char...|
+-------+--------------------+



In [15]:
from pyspark.sql.functions import explode
df.select(df['name'],df['items'],explode(df['items']).alias('explode_column'),df.items[0],df.items[1]).show()

+-------+--------------------+--------------+----------+--------+
|   name|               items|explode_column|  items[0]|items[1]|
+-------+--------------------+--------------+----------+--------+
|  Alice|     [Laptop, Mouse]|        Laptop|    Laptop|   Mouse|
|  Alice|     [Laptop, Mouse]|         Mouse|    Laptop|   Mouse|
|    Bob|             [Phone]|         Phone|     Phone|    NULL|
|Charlie|  [Tablet, Keyboard]|        Tablet|    Tablet|Keyboard|
|Charlie|  [Tablet, Keyboard]|      Keyboard|    Tablet|Keyboard|
|  David|           [Monitor]|       Monitor|   Monitor|    NULL|
|   Emma|[Headphones, Char...|    Headphones|Headphones| Charger|
|   Emma|[Headphones, Char...|       Charger|Headphones| Charger|
+-------+--------------------+--------------+----------+--------+



In [30]:

from pyspark.sql.types import StringType, StructField, StructType,ArrayType, IntegerType, StringType

data1=[(101, "Alice", ("Laptop", "Mouse"), 1200, "2024-02-15", "Shipped"),
    (102, "Bob", ("Phone",""), 800, "2024-02-16", "Pending"),
    (103, "Charlie", ("Tablet", "Keyboard"), 600, "2024-02-17", "Delivered"),
    (104, "David", ("Monitor",""), 300, "2024-02-18", "Canceled"),
    (105, "Emma", ("Headphones", "Charger"), 150, "2024-02-19", "Shipped")]

schema=StructType([
    StructField('id',IntegerType(),True),
    StructField('name',StringType(),True),
    StructField('items',StructType([
        StructField('item1',StringType(),True),
        StructField('item2',StringType(),True)
    ]),True),
    StructField('amount',IntegerType(),True),
    StructField('orderDate',StringType(),True),
    StructField('orderStatus',StringType(),True)
])

df1 = spark.createDataFrame(data1,schema)
df1.show()
df1.printSchema()

df1.select(df1.items.item1,df1.items.item2).show()

+---+-------+--------------------+------+----------+-----------+
| id|   name|               items|amount| orderDate|orderStatus|
+---+-------+--------------------+------+----------+-----------+
|101|  Alice|     {Laptop, Mouse}|  1200|2024-02-15|    Shipped|
|102|    Bob|           {Phone, }|   800|2024-02-16|    Pending|
|103|Charlie|  {Tablet, Keyboard}|   600|2024-02-17|  Delivered|
|104|  David|         {Monitor, }|   300|2024-02-18|   Canceled|
|105|   Emma|{Headphones, Char...|   150|2024-02-19|    Shipped|
+---+-------+--------------------+------+----------+-----------+

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- items: struct (nullable = true)
 |    |-- item1: string (nullable = true)
 |    |-- item2: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- orderDate: string (nullable = true)
 |-- orderStatus: string (nullable = true)

+-----------+-----------+
|items.item1|items.item2|
+-----------+-----------+
|     Laptop|  

In [31]:
df1.select(df1['items.item1'],df1['items.item2']).show()

+----------+--------+
|     item1|   item2|
+----------+--------+
|    Laptop|   Mouse|
|     Phone|        |
|    Tablet|Keyboard|
|   Monitor|        |
|Headphones| Charger|
+----------+--------+



In [33]:
df1.select(col('items.item1'),col('items.item2'),df1.name).show()

+----------+--------+-------+
|     item1|   item2|   name|
+----------+--------+-------+
|    Laptop|   Mouse|  Alice|
|     Phone|        |    Bob|
|    Tablet|Keyboard|Charlie|
|   Monitor|        |  David|
|Headphones| Charger|   Emma|
+----------+--------+-------+



In [34]:
from pyspark.sql.functions import lit

df1.select('id','name',lit('bala').alias('test')).show()

+---+-------+----+
| id|   name|test|
+---+-------+----+
|101|  Alice|bala|
|102|    Bob|bala|
|103|Charlie|bala|
|104|  David|bala|
|105|   Emma|bala|
+---+-------+----+

